In [2]:
import import_ipynb
import threading
import numpy as np

from class_AEdriver_rotation import AEdriver, stable_rotation
from eg_encoder_daq_refact import processor, log_listener, daq, start_gui, stop_writer

def reset(COM):
    try:
        driver = AEdriver(COM)
    except:
        pass
    driver.stop()
    driver.initialize()
    driver.ser.close()

dset shape  = (0, 3)
dset maxshape = (None, 3)


In [3]:
pwd

'/Users/taiseiiwagaki/Jupyter Notebook/BBM2 cryostat data/2025'

In [4]:
COM='COM5'
reset(COM)   # run the first cell

Exception ignored in: <function AEdriver.__del__ at 0x1143218a0>
Traceback (most recent call last):
  File "<string>", line 37, in __del__
AttributeError: 'AEdriver' object has no attribute 'ser'


UnboundLocalError: cannot access local variable 'driver' where it is not associated with a value

In [5]:
#  1Hz

i = 2
rot_rpm = [30, 46, 60, 120]
rot_freq = [0.5, 0.766, 1., 2.0]
curr_freq = 36*rot_freq[i]          # 72 SmCO magnet, -> 36 pole pairs
curr_pps_fin = curr_freq*1000.      # current pulse per second finish


initial_curr = 45                   # initial_curr = % of 1A
                                    # Conversion factor: (current = initial_curr*10 mA) 
                                    # If input 200mA, write 20 here. Input 300mA, write 30.


pps = 100                           # Initial current pulse per second
pps_fin = int(curr_pps_fin)         # Current pulse per second for stable rotation, minimum current finding
step = 5 # 100                    # Acceleration of pps, normally step=100 for 0.26 coil wire, 50 for 0.5 mm coil wire


ppsps = 1000 #pps*step              #  10000     # Maximum acceleration, waiting step for pps per second => ppsps = pps*step
rotshift_time = 1 # Sleep time for shifting the pps
stablerot_duration = 3600*2             # [s] Time for stable rotation
excess_spindown_time = 3600 #1800    # time for spindown, used to be 30'


print("=====================================================================================")
print('Rotational frequency                           : ', rot_freq[i], '[Hz]')
print('Current pulse per second to finish acceleration: ', curr_pps_fin )
print('Initial current                                : ', '%1f'%(initial_curr*10) , '[mA]')
print('Initial Pulse per second                       : ',pps )
print('Maximum acceleration step                      : ',ppsps )
print('Rotational shift after every pps               : ',rotshift_time , '[s]')
print('Step                                           : ', step)
print('Stable rotation duration                       : ', stablerot_duration/60, '[min]')
print('excess time before stop the rotor command      : ', excess_spindown_time/60, '[min]')
print("======================================================================================")

Rotational frequency                           :  1.0 [Hz]
Current pulse per second to finish acceleration:  36000.0
Initial current                                :  450.000000 [mA]
Initial Pulse per second                       :  100
Maximum acceleration step                      :  1000
Rotational shift after every pps               :  1 [s]
Step                                           :  5
Stable rotation duration                       :  120.0 [min]
excess time before stop the rotor command      :  60.0 [min]


In [6]:
# 0: clockwise, 1: counter clockwise
test = stable_rotation(pps, pps_fin, step, ppsps, initial_curr, rotshift_time, stablerot_duration, excess_spindown_time, rot=1)
thread2 = threading.Thread(target=test.rotation, daemon=True)
#thread2.setDaemon(True) rewrite by RA using prefarable coding
thread2.start()
thread2.join()

if __name__ == "__main__":
    threading.Thread(target=log_listener, daemon=True).start()  # start log listener

    daq_th = threading.Thread(target=daq, daemon=True)
    proc_th = threading.Thread(target=processor, daemon=True)

    # motor driver
    run = stable_rotation(pps, pps_fin, step, ppsps, initial_curr, rotshift_time, stablerot_duration, excess_spindown_time, rot=1)
    motor_th = threading.Thread(target=run.rotation, daemon=True)

    # start job
    daq_th.start()
    proc_th.start()
    motor_th.start()

    start_gui()  # blocks until the user closes the window or timer expires

    # join threads and exit
    stop_writer.set()
    

    daq_th.join()
    proc_th.join()
    motor_th.join()

All process start time 2025-05-27 16:30:57.856808


AttributeError: module 'serial' has no attribute 'Serial'